In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas_profiling

In [ ]:
d = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")

In [ ]:
d.Glucose = d.Glucose.replace(0,np.nan)
d.BloodPressure = d.BloodPressure.replace(0,np.nan)
d.SkinThickness = d.SkinThickness.replace(0,np.nan)
d.Insulin = d.Insulin.replace(0,np.nan)
d.BMI = d.BMI.replace(0,np.nan)
d.DiabetesPedigreeFunction = d.DiabetesPedigreeFunction.replace(0,np.nan)
d.Age = d.Age.replace(0,np.nan)

In [ ]:
d.count()

In [ ]:
d = d.fillna(value = {'Glucose':d.Glucose.mean(), 'BloodPressure':d.BloodPressure.mean(), 'SkinThickness':d.SkinThickness.mean(), 'Insulin':d.Insulin.mean(), 'BMI':d.BMI.mean()})

In [ ]:
d.count()

In [ ]:
sns.pairplot(d, hue="Outcome")

In [ ]:
X = d.drop(['Outcome'], axis=1)
y = d.Outcome

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
y_lr = lr.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

In [ ]:
print(confusion_matrix(y_test,y_lr))

In [ ]:
pd.crosstab(y_test, y_lr, rownames=['Reel'], colnames=['Prediction'], margins=True)

In [ ]:
print(accuracy_score(y_test,y_lr))

In [ ]:
print(classification_report(y_test, y_lr))

In [ ]:
probas = lr.predict_proba(X_test)

In [ ]:
print(probas)

In [ ]:
dprobas = pd.DataFrame(probas,columns=['proba_0','proba_1'])
dprobas['y'] = np.array(y_test)

In [ ]:
dprobas

In [ ]:
plt.figure(figsize=(10,10))
sns.distplot(1-dprobas.proba_0[dprobas.y==0], bins=50)
sns.distplot(dprobas.proba_1[dprobas.y==1], bins=50)

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,probas[:, 1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print (roc_auc)

In [ ]:
plt.figure(figsize=(12,12))
plt.title('Receiver Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')        # plus mauvaise courbe
plt.plot([0,0,1],[0,1,1],'g:')     # meilleure courbe
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

In [ ]:
from sklearn import ensemble
r = ensemble.RandomForestClassifier()
r.fit(X_train, y_train)
y_r = r.predict(X_test)

In [ ]:
print(accuracy_score(y_test,y_r))

In [ ]:
print(classification_report(y_test, y_r))

In [ ]:
c = confusion_matrix(y_test, y_r)
print(c)

In [ ]:
pd.crosstab(y_test, y_r, rownames=['Reel'], colnames=['Prediction'], margins=True)

In [ ]:
from sklearn import model_selection

In [ ]:
param_grid = {
              'n_estimators': [10, 100, 500],
              'min_samples_leaf': [1, 20, 50]
             }
estimator = ensemble.RandomForestClassifier()
r_gs = model_selection.GridSearchCV(estimator, param_grid)
r_gs.fit(X_train, y_train)
print(r_gs.best_params_)

In [ ]:
r2 = r_gs.best_estimator_
y_r2 = r2.predict(X_test)
print(classification_report(y_test, y_r2))

In [ ]:
importances = r2.feature_importances_
indices = np.argsort(importances)

In [ ]:
plt.figure(figsize=(8,5))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), X_train.columns[indices])
plt.title('Importance des caracteristiques')

In [ ]:
!pip install xgboost

In [ ]:
import xgboost as XGB
xgb  = XGB.XGBClassifier()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
cm = confusion_matrix(y_test, y_xgb)
print(cm)
print(classification_report(y_test, y_xgb))